In [1]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import img_to_array
import pyttsx3
import enchant  # Ensure you have this installed

# Load the pre-trained model
model = load_model('C:/Users/Likitha/Desktop/New folder/model.h5')

# Define the labels (customize these according to your model's output)
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']

# Initialize Text to Speech engine
tts_engine = pyttsx3.init()

# Initialize the spell checker
spell_checker = enchant.Dict("en_US")

# Function to convert sign language to text
def sign_to_text(frame):
    roi = cv2.resize(frame, (64, 64))
    roi = roi.astype("float") / 255.0
    roi = img_to_array(roi)
    roi = np.expand_dims(roi, axis=0)

    # Predict the class
    predictions = model.predict(roi)
    max_index = np.argmax(predictions[0])
    predicted_label = labels[max_index]
    
    return predicted_label

# Function to convert text to speech
def text_to_speech(text):
    tts_engine.say(text)
    tts_engine.runAndWait()

# Function to start the video feed
def start_video_feed():
    global cap, update_frame
    cap = cv2.VideoCapture(0)
    update_frame()

# Function to stop the video feed
def stop_video_feed():
    cap.release()

# Function to clear the predicted text
def clear_text():
    text_label.config(text="")
    sentence_textbox.delete('1.0', tk.END)

# Function to read text from the text box and convert it to speech
def convert_text_to_speech():
    text = sentence_textbox.get("1.0", tk.END+"-1c").strip()
    if text:
        if spell_checker.check(text):
            text_to_speech(text)
        else:
            suggested_corrections = spell_checker.suggest(text)
            if suggested_corrections:
                corrected_text = suggested_corrections[0]
            else:
                corrected_text = text
            sentence_textbox.delete("1.0", tk.END)
            sentence_textbox.insert(tk.END, corrected_text)
            text_to_speech(corrected_text)


# Initialize the main window
root = tk.Tk()
root.title("Sign Language Classifier")

# Create a notebook to hold the tabs
notebook = ttk.Notebook(root)
notebook.pack(expand=True, fill='both')

# Create the first tab
frame1 = ttk.Frame(notebook)
notebook.add(frame1, text="Classification")

# Create a frame for the video feed
frame_video = tk.Frame(frame1)
frame_video.pack()

# Create a canvas to display the video feed
canvas = tk.Canvas(frame_video, width=640, height=480)
canvas.pack()

# Create a label to display the classified text
text_label = tk.Label(frame1, text="", font=("Helvetica", 16))
text_label.pack()

# Create a text box to keep track of the sentence
sentence_textbox = tk.Text(frame1, height=5, width=50, font=("Helvetica", 14))
sentence_textbox.pack()

# Create a button to trigger text-to-speech conversion
tts_button = tk.Button(frame1, text="Text to Speech", command=convert_text_to_speech)
tts_button.pack()

# Create an exit button
exit_button = tk.Button(frame1, text="Exit", command=root.quit)
exit_button.pack(side=tk.RIGHT)

# Create a button to start the video feed
start_button = tk.Button(frame1, text="Start", command=start_video_feed)
start_button.pack(side=tk.RIGHT)

# Create a button to stop the video feed
stop_button = tk.Button(frame1, text="Stop", command=stop_video_feed)
stop_button.pack(side=tk.RIGHT)

# Create a button to clear the predicted text
clear_button = tk.Button(frame1, text="Clear", command=clear_text)
clear_button.pack(side=tk.RIGHT)

# Create the second window for the ROI frame
roi_window = tk.Toplevel(root)
roi_window.title("ROI Frame")

# Create a canvas to display the ROI frame
canvas_roi = tk.Canvas(roi_window, width=300, height=300)
canvas_roi.pack()

# Function to update the frame
def update_frame():
    ret, frame = cap.read()
    if not ret:
        return
    
    frame = cv2.flip(frame, 1)  # Flip the frame horizontally

    # Get the region of interest for classification (customize the ROI if needed)
    roi = frame[100:400, 100:400]
    cv2.rectangle(frame, (100, 100), (400, 400), (0, 255, 0), 2)

    # Predict the sign language
    predicted_label = sign_to_text(roi)
    text_label.config(text=predicted_label)

    # Append the predicted label to the sentence text box if it's not 'nothing'
    if predicted_label == 'space':
        sentence_textbox.insert(tk.END, ' ')
    elif predicted_label == 'del':
        current_text = sentence_textbox.get("1.0", tk.END).strip()
        current_text = current_text[:-1]
        sentence_textbox.delete("1.0", tk.END)
        sentence_textbox.insert(tk.END, current_text)
    elif predicted_label != 'nothing':
        sentence_textbox.insert(tk.END, predicted_label)

    # Convert the frame to a format that Tkinter can display
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    img_tk = ImageTk.PhotoImage(image=img)

    # Update the canvas with the new frame
    canvas.create_image(0, 0, anchor=tk.NW, image=img_tk)
    canvas.image = img_tk

    # Process the ROI for the second window
    roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    roi_blur = cv2.GaussianBlur(roi_gray, (5, 5), 0)
    _, roi_thresh = cv2.threshold(roi_blur, 60, 255, cv2.THRESH_BINARY_INV)

    # Convert the ROI to a format that Tkinter can display
    img_roi = Image.fromarray(roi_thresh)
    img_roi_tk = ImageTk.PhotoImage(image=img_roi)

    # Update the canvas with the new ROI frame
    canvas_roi.create_image(0, 0, anchor=tk.NW, image=img_roi_tk)
    canvas_roi.image = img_roi_tk

    # Call this function again after 10 milliseconds
    root.after(10, update_frame)

# Start the Tkinter main loop
root.mainloop()


1/1 [==============================] - 0s 35ms/step
